# Create a function to create animation of the particles moving

In [16]:
from random import random

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ParticleSwarm import ParticleSwarmOptimization, GenerationSnapshot

def animate_snapshots(snapshots:list[GenerationSnapshot], interval=200):
    fig, ax = plt.subplots(figsize=(7,7))

    # House scatter
    house_scatter = ax.scatter([], [], c='red', marker='s', label='Houses')
    # Particle scatter
    particle_scatter = ax.scatter([], [], c='blue', marker='o', label='Particles')

    # Initialize quiver with first snapshot
    snap0 = snapshots[0]
    px0 = np.array([p.getPosition().getX() for p in snap0.particles])
    py0 = np.array([p.getPosition().getY() for p in snap0.particles])
    vx0 = np.array([p.getVelocity().getX() for p in snap0.particles])
    vy0 = np.array([p.getVelocity().getY() for p in snap0.particles])
    quiver = ax.quiver(px0, py0, vx0, vy0, angles='xy', scale_units='xy', scale=1, color='green')

    best_scatter = ax.scatter([], [], c='yellow', marker='*', label='Best')

    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)
    ax.axis('equal')
    ax.legend()

    # Compute axis limits based on all snapshots
    all_x = []
    all_y = []
    for snap in snapshots:
        all_x.extend([p.getPosition().getX() for p in snap.particles])
        all_y.extend([p.getPosition().getY() for p in snap.particles])
        all_x.extend([h.getPosition().getX() for h in snap.houses])
        all_y.extend([h.getPosition().getY() for h in snap.houses])
    ax.set_xlim(min(all_x)-1, max(all_x)+1)
    ax.set_ylim(min(all_y)-1, max(all_y)+1)

    # ---------------- UPDATE FUNCTION ----------------
    def update(i):
        snap : GenerationSnapshot = snapshots[i]

        # Particle positions
        ps = [[p.getPosition().getX(), p.getPosition().getY()] for p in snap.particles]

        # Particle velocities
        vx = np.array([p.getVelocity().getX() for p in snap.particles])
        vy = np.array([p.getVelocity().getY() for p in snap.particles])

        # House positions
        hx = np.array([h.getPosition().getX() for h in snap.houses])
        hy = np.array([h.getPosition().getY() for h in snap.houses])
        
        # Update scatters
        house_scatter.set_offsets(np.c_[hx, hy])
        particle_scatter.set_offsets(ps)

        if (snap.best):
            best_scatter.set_offsets([snap.best.position.x, snap.best.position.y])
        else:
            best_scatter.set_offsets([])

        # Update quiver arrows
        quiver.set_offsets(ps)
        quiver.set_UVC(vx, vy)

        ax.set_title(f"Generation {snap.gennum}")
        return house_scatter, particle_scatter, quiver, best_scatter

    anim = FuncAnimation(
        fig,
        update,
        frames=len(snapshots),
        interval=interval,
        blit=False
    )

    plt.close(fig)
    return anim


# Create the map with randomly generated houses

In [17]:
Swarm = ParticleSwarmOptimization(40)

## Calculate X Particles for Y Generations

In [18]:
gens : list = Swarm.calculate(10, 70)

# Display animation

In [19]:
from IPython.display import HTML
anim = animate_snapshots(gens, interval=100)
HTML(anim.to_jshtml())

Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


## Save animation

In [24]:
from random import randint
anim.save(f"../media/example{randint(1000, 9999)}.gif")

MovieWriter ffmpeg unavailable; using Pillow instead.
